In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import json
import pandas
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) 
#if you wnt to use your gpu and this command doesnt list it, close jupyter session (ctrl+c in cmd) and this notebook
#copy all 3 commands and launch jupyter notebook again
#if it still does not list check hardware and software compatibility

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7262129719995881890
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4934185779
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13335580058563771292
physical_device_desc: "device: 0, name: GeForce GTX 980 Ti, pci bus id: 0000:01:00.0, compute capability: 5.2"
]


In [2]:
directory = r'D:\lyrics_gen\rap_lyrics\rap_lyrics_final\lyrics' #insert your path to lyrics

'''for filename in os.listdir(directory):
    print(filename)'''
    
lyrics = []
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        path = os.path.join(directory, filename)
        with open(path) as f:
            try:
                data = json.load(f)
                lyrics.append(data["Lyrics"])
            except: continue
    else: continue
        

In [3]:
len(lyrics)

1255

In [4]:
flat_lyrics = []
for song in lyrics:
    for verse in song:
        for word in verse:
            for letter in word:
                flat_lyrics.append(letter)

In [5]:
len(flat_lyrics)

4117866

In [6]:
# The unique characters in the file
good = ['\n', ' ', '!', '"', '&', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [7]:
flat_lyrics_clean = []
for letter in flat_lyrics: 
    if letter in good: flat_lyrics_clean.append(letter)
    else: continue

In [8]:
len(flat_lyrics_clean)

4080064

In [9]:
vocab = sorted(set(flat_lyrics_clean))
print ('{} unique characters'.format(len(vocab)))
print(vocab)

76 unique characters
['\n', ' ', '!', '"', '&', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [10]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in flat_lyrics_clean])

In [11]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '0' :   9,
  '1' :  10,
  '2' :  11,
  '3' :  12,
  '4' :  13,
  '5' :  14,
  '6' :  15,
  '7' :  16,
  '8' :  17,
  '9' :  18,
  ':' :  19,
  ...
}


In [12]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(flat_lyrics_clean[:13]), text_as_int[:13]))

['\n', 'N', 'o', 'b', 'o', 'd', 'y', ' ', 'p', 'r', 'a', 'y', ' '] ---- characters mapped to int ---- > [ 0 35 64 51 64 53 74  1 65 67 50 74  1]


In [13]:
# The maximum length sentence we want for a single input in characters
seq_length = 1024
examples_per_epoch = len(flat_lyrics_clean)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])



N
o
b
o
d
y
 
p
r


In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"\nNobody pray for me\nIt's been that day for me\nWay Yeah, yeah!\n\n\nAyy, I remember syrup sandwiches and crime allowances\nFinesse a nigga with some counterfeits, but now Im countin' this\nParmesan where my accountant lives, in fact I'm downin this\nD'USS with my boo bae tastes like Kool-Aid for the analysts\nGirl, I can buy yo' ass the world with my paystub\nOoh, that pussy good, won't you sit it on my taste bloods?\nI get way too petty once you let me do the extras\nPull up on your block, then break it down: we playin' Tetris\nA.M. to the P.M., P.M. to the A.M., funk\nPiss out your per diem, you just gotta hate 'em, funk\nIf I quit your BM, I still ride Mercedes, funk\nIf I quit this season, I still be the greatest, funk\nMy left stroke just went viral\nRight stroke put lil' baby in a spiral\nSoprano C, we like to keep it on a high note\nIt's levels to it, you and I know\n\n\nBitch, be humble Hol up, bitch\nSit down Hol up, lil', hol up, lil' bitch\nBe humble Hol' up, bitch\nSit d

In [15]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [16]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "\nNobody pray for me\nIt's been that day for me\nWay Yeah, yeah!\n\n\nAyy, I remember syrup sandwiches and crime allowances\nFinesse a nigga with some counterfeits, but now Im countin' this\nParmesan where my accountant lives, in fact I'm downin this\nD'USS with my boo bae tastes like Kool-Aid for the analysts\nGirl, I can buy yo' ass the world with my paystub\nOoh, that pussy good, won't you sit it on my taste bloods?\nI get way too petty once you let me do the extras\nPull up on your block, then break it down: we playin' Tetris\nA.M. to the P.M., P.M. to the A.M., funk\nPiss out your per diem, you just gotta hate 'em, funk\nIf I quit your BM, I still ride Mercedes, funk\nIf I quit this season, I still be the greatest, funk\nMy left stroke just went viral\nRight stroke put lil' baby in a spiral\nSoprano C, we like to keep it on a high note\nIt's levels to it, you and I know\n\n\nBitch, be humble Hol up, bitch\nSit down Hol up, lil', hol up, lil' bitch\nBe humble Hol' up,

In [17]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 0 ('\n')
  expected output: 35 ('N')
Step    1
  input: 35 ('N')
  expected output: 64 ('o')
Step    2
  input: 64 ('o')
  expected output: 51 ('b')
Step    3
  input: 51 ('b')
  expected output: 64 ('o')
Step    4
  input: 64 ('o')
  expected output: 53 ('d')


In [18]:
#!You may need to lower batch and buffer sized if your pc is a little bit slower!
#if thats the case try lowering batch size to 64 and buffer size to 10000 or even lower


# Batch size
BATCH_SIZE = 75

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 50000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((75, 1024), (75, 1024)), types: (tf.int32, tf.int32)>

In [19]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [21]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(75, 1024, 76) # (batch_size, sequence_length, vocab_size)


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (75, None, 256)           19456     
_________________________________________________________________
gru (GRU)                    (75, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (75, None, 76)            77900     
Total params: 4,035,660
Trainable params: 4,035,660
Non-trainable params: 0
_________________________________________________________________


In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [25]:
sampled_indices

array([48, 44, 61, ..., 11, 38,  1], dtype=int64)

In [26]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "rip, drip\nCame through drippin' drip, drip\nCame through drippin' drip, drip\nDiamonds on my wrist, they drippin' ice!\nCame through drippin' drip, drip\nCame through drippin' drip, drip\nCame through drippin' drip, drip\nDiamonds on my wrist, they drippin' ice!\n\n\nGive me little something to remember Cardi!\nTryna make love in a Sprinter yeah\nQuick to drop a nigga like Kemba go\nLookin' like a right swipe on Tinder woo\nShit on these hoes shit\nLight up my wrist on these hoes wrist\nNow I look down on these bitches down\nI feel like I'm on stilts on these hoes woo\nFuck ya' baby daddy right now right now\nAnna Mae, got cake by the pound pound\nGo down, eat it up, don't drown\nMac n' cheese in the bowl, how it sound? sound\nI got that gushy\nYeah that's a fact, but I never been pussy\nI've been that bitch since pajamas with footies\nWon MVP, and I'm still a rookie, like woo\nI gotta work on my anger ayy\nMight kill a bitch with my fingers ayy\nI gotta stay outta Gucci woo

In [27]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (75, 1024, 76)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.33152


In [28]:
#if you want to use our model in stead of training one skip straigh to loading model
model.compile(optimizer='adam', loss=loss)

In [29]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [30]:
EPOCHS=100

In [31]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 53 steps
Epoch 1/100
53/53 [==============================] - 41s 776ms/step - loss: 3.2525
Epoch 2/100
53/53 [==============================] - 40s 756ms/step - loss: 2.4102
Epoch 3/100
53/53 [==============================] - 40s 755ms/step - loss: 2.2091
Epoch 4/100
53/53 [==============================] - 40s 754ms/step - loss: 2.0301
Epoch 5/100
53/53 [==============================] - 40s 756ms/step - loss: 1.8809
Epoch 6/100
53/53 [==============================] - 40s 753ms/step - loss: 1.7631
Epoch 7/100
53/53 [==============================] - 40s 753ms/step - loss: 1.6714
Epoch 8/100
53/53 [==============================] - 40s 753ms/step - loss: 1.6006
Epoch 9/100
53/53 [==============================] - 40s 754ms/step - loss: 1.5434
Epoch 10/100
53/53 [==============================] - 40s 755ms/step - loss: 1.4963
Epoch 11/100
53/53 [==============================] - 40s 755ms/step - loss: 1.4573
Epoch 12/100
53/53 [==============================] - 40s 755ms/st

In [32]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_100'

In [33]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            19456     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 76)             77900     
Total params: 4,035,660
Trainable params: 4,035,660
Non-trainable params: 0
_________________________________________________________________


In [28]:
#you can load provided model using this command
#if you have trainded your own model skip this step
model = tf.keras.models.load_model('good_model_100_epochs.h5', compile=False) 

In [29]:
def generate_text(model, start_string, num_generate, temperature):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = num_generate

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  # You can try experimenting with values between 0 and 2 but values lower than 0.6 are very likely to just copy existing lyrics
  # and values higher than 1.5 are going to be random gibberish (remember to use . not ,)
  temperature = temperature

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [30]:
print(generate_text(model, start_string=u"Smoke ", num_generate = 1500, temperature = 0.3))

Smoke and raised your block
So image is down with it should be anywhere...
You ain't never told no lie, you ain't never told no lie
Real niggas say word
You ain't never told no lie, you ain't never told no lie
Real niggas say word
You ain't never told no lie, you ain't never told no lie
Real niggas say word
You ain't never told no lie, you ain't never told no lie
Real niggas say word
You ain't never told no lie, you ain't never told no lie
Real niggas say word
You ain't never told no lie, you ain't never told no lie
Real niggas say word
You ain't never told no lie, you ain't never told no lie
Real niggas say word
You ain't never told no lie, you ain't never told no lie
Real niggas say word
You ain't never told no lie, you ain't never told no lie
Real niggas say word
You ain't never told no lie, you ain't never told no lie
Real niggas say word
You ain't never told no lie, you ain't never told no lie
Real niggas say word
You ain't never told no lie, you ain't never told no lie
Real nigga

In [31]:
print(generate_text(model, start_string=u"Smoke ", num_generate = 1500, temperature = 0.7))

Smoke an apple!
Somebody need mat fighting out they magning
Cause ain't no broke by ain't mine
She want more than put ya would you still don't know if
If I'll ever listen to my mama
I don't really hang with Time


What's the difference between det in?
Heard you the shit that you came with, who you came in this hotel room
You may get a bit
When you make it, eat on that pain
I'm sorry I made my dick for just a lot of constants
But changes dispose blunts, three hoes stucked
She tried to act like they watch Thatless game
What you thought? You want beef? I like the way you be loated
Cause it's all for the kind of bitches, they don't love you no more
They don't love you no more
Don't fly, I don't get signed by 12
Bitch conted upon Double Rich Rose alive
Wait, honey, choose to success so its gone be a freak
Cause niggas talkin' bout piney gon' hate
But I know that I know it, I know I chose ain't nothin' better
Now it's time to change, sometimes it dip, make it dip, make it dip, make it dip Ay

In [32]:
print(generate_text(model, start_string=u"Smoke ", num_generate = 1500, temperature = 1.1))

Smoke and robbin' bitch
'Cause you murdered over like a T-Rex Ro
Niggas got firently like, "bitch" deep and formally talkin'
"What the fuck, go watch, today 5 mill
He said "Yo baby tell 'em, we don't take rap lame
Yeah aight, I'm 'bout that Bambi Ahhhh
My life is finally from Hova Lito die USNeV
Ebus taste on that cookie
Fucking rappin' about Zooony Pity
I was only missin' on the level to add-to-ta
No one calling us while Jevis holler-bain


No we are even got nothing to lost


Don't stop, pop that Hoffa, I'm violence
If Brookly Winton, I was toling on rap
And when you suck it, I bust, real shit
It's a day, she look at her drip-drip
Diamonds gotta find them noish, no way, no o-to-day
I'll put my time the watch, I'm on a new levovat
We came regular, never chase a fuck nigga
J my nigga Ema usual Long Beachts


You ain't thinkin' 'bout your issue?
Whole squad, just is cutter seats
No cryin'
To black and white, hop Davis, my cock
You know I be thuggin', on my hood shit
Niggas still love me

In [33]:
print(generate_text(model, start_string=u"Smoke ", num_generate = 1500, temperature = 1.5))

Smoke anyo sex, no a tripy
And white to Aufi superation, hoping about M's in Vra
Song Lord Catantsey hurt
I know that yo' musf lucky Yuah
Mine & now-a-umitate its begging for more
Mysic musicy chalon feet gettin' "Master degan" scared can't get your dons
Me, low at titties shit
Pull up to the biggest life for now Wittable, I put dounging Cash stoner
Flip the safe La, Ear, had to rusy as we used to weave
Everyone falls in love You-usin! Erguent!
GePreclet I make a few school
New Billy Coby Lady's blame
Sezen dependin' workin' out of coochie, auf I di live, it won't snatch back
Hor pay attention, Im ran into room
I just pop more, a Nigga, I'm self-toast Woo
Lost is you riding?
Don't you do to me no ass, lo-do!-G, by by
Phycalthem
Moving hang you out once, declini came


All of y'all niggas cop the hat to fake it
Man, got attention, because, i-huntry Junky, come on and fuckin' support actin' Whoo
Feet Ooh-Ooh, I put on, fuck this pussy, god all propanx
Merodadone


Ohhhh, nigga, where her

In [34]:
#you can see that different temperatures produce very different texts
#now we can experiment with different words

In [35]:
print(generate_text(model, start_string=u"I ", num_generate = 1500, temperature = 1.05))

I got the re-up
These niggas I done popped you, nigga
Anybody wanna talk about whe the cooling Mochers
Might easy and watchin' that WKA, we got some baity and duct they show
You're on your little judge, chop niggas boolin' for me Right
Ooh, don't stop


You wanna say that what you wanted was the one bitch, I'm in chastings
And I'm gettin' more than that pat, mmm, ah, mmm
I got please, that a best friend, best
Drake said if we kicked a stick up and we can see if I tried
Ein, me know how we became feel a rap
That I know when the world fall for frunted with a bunch of gross
In my players playing no pistols


A fuck nigga, that's that shit I don't like
Don't told me that's right, shit is miperianton
In my life I can't let her ass in a Frendi
Fed a flip with another liter, and the idea you dont nobody raised
You see my damn time, the most started with you
Locus for the fake, juggler pinnace
Dip 'em in a corn ain't enough for me
Ridin' 'round to talk that New York Pet, pap's over-
I do the t

In [36]:
print(generate_text(model, start_string=u"Ballin ", num_generate = 1500, temperature = 1.05))

Ballin knew Id makes me smile


What's fire in? Don't get nuded off as zent B
You caught up in cover on it with you, and now you out here made of glass up on the floors
Group please, I be racin'
Just get enough, just move they bitches I can't just leavin' you
Never beat the peraps off the end is inno us
Theres nothin water beats out a headless
Everybody know you butterful chicken giff a team
You were at your name
You think I didn't be the one to talk about it?
Which crack to seed up great
Fuckin' with my clique, blow up that part
Talkin' pow-pow-pop like I'm trying for me
You a savage, hold up, yo
The one with a real niggas making my black hole
To all the realishing, mangu made until you're spilling
And when I OG and I be goin' real, O's
You ain't nothin' but a bust down
Toes a little effer better, but still made a hat nigga
The first and friends who will tell
Them D's enough too many shatter and a man had the hate
You gotta run up on the curb, it's like a hail that shit
Don't hit your

In [37]:
print(generate_text(model, start_string=u"Yeah ", num_generate = 1500, temperature = 1.05))

Yeah when I created this, I wanna be changing the smilerd
Edjust it cum Im doin went and ride in pecados
In the club, rotarchip
Yo, Pap, ya brand new Benz tell
But my daughter too breathesh or throwed bad back
Was the same people interfering up an advised, I always get her tough the camera
But I know she wanna miss Nah
I swerve, but I don't know
But, shit, I'm still one who left me I ain't know you like michael brrr
You never was so made the nigga having battle rap even off
Niggas stupid with us, we notice this future; don't wanna cop it?
My dick and you'll turn a nigga dope like my nigga


It's how you think alone... down
Hold on 'em, shittling dick
Triple black trail slide Runnin' swang
Hage neck, my bands right, right, left, left, right


Girl!


Mix deuce on my sneakers like boomeno Yeah
That's why you show me why you dyin' through the naicious?
Sever you don't really duck it behold
Shoulda keep a lot of business of speedboat
She a big girl, dog
I'm gon' shine ain't always gon' tie